# **1. Import Library**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix, classification_report

# **2. Memuat Dataset**

## Tampilkan Data

In [2]:
df_cleaned = pd.read_csv('https://raw.githubusercontent.com/Sulbae/SMSML_Anggun-Sulis-Setyawan/refs/heads/main/Eksperimen_SML_Anggun-Sulis-Setyawan/Preprocessing/water_potability_preprocessing')

df_cleaned.head()

HTTPError: HTTP Error 404: Not Found

# **3. Persiapan Data**

## Pembagian Data Pelatihan

In [ ]:
def split_data(data, test_size=0.25, random_state=42):
    X = data.drop(columns='Potability', axis=1)
    y = data['Potability']
    return train_test_split(X, y, test_size=test_size, stratify=y, random_state=random_state)

In [ ]:
X_train, X_test, y_train, y_test = split_data(df_cleaned, test_size=0.25, random_state=42)

print(f"X_train Shape: {X_train.shape}")
print(f"y_train Shape: {y_train.shape}")
print(f"X_test Shape: {X_test.shape}")
print(f"y_test Shape: {y_test.shape}")

X_train Shape: (2457, 9)
y_train Shape: (2457,)
X_test Shape: (819, 9)
y_test Shape: (819,)


## Pelatihan Model

In [ ]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    results = []

    # RF Model
    rf_pipeline = Pipeline([
        ('model', RandomForestClassifier(n_estimators=100, random_state=42))
    ])
    rf_pipeline.fit(X_train, y_train)
    y_pred_rf = rf_pipeline.predict(X_test)
    rf_report = pd.DataFrame(classification_report(y_test, y_pred_rf, output_dict=True)).T
    rf_report['model'] = 'Random Forest'
    results.append(rf_report)

    # SVC Model
    svc_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', SVC(kernel='rbf', probability=True, random_state=42))
    ])
    svc_pipeline.fit(X_train, y_train)
    y_pred_svc = svc_pipeline.predict(X_test)
    svc_report = pd.DataFrame(classification_report(y_test, y_pred_svc, output_dict=True)).T
    svc_report['model'] = 'SVC'
    results.append(svc_report)

    combined = pd.concat(results).reset_index().rename(columns={'index' : 'metrics'})
    comparison = combined[combined['metrics'].isin(['0', '1', 'accuracy', 'macro avg', 'weighted avg'])]
    return comparison[['model', 'metrics', 'precision', 'recall', 'f1-score', 'support']].round(2)

In [ ]:
models_training = train_and_evaluate_models(X_train, X_test, y_train, y_test)

In [ ]:
models_training.sort_values(by='metrics')

model       metrics  precision  recall  f1-score  support
0  Random Forest             0       0.67    0.88      0.76   499.00
5            SVC             0       0.67    0.93      0.78   499.00
1  Random Forest             1       0.64    0.32      0.43   320.00
6            SVC             1       0.71    0.27      0.39   320.00
2  Random Forest      accuracy       0.66    0.66      0.66     0.66
7            SVC      accuracy       0.67    0.67      0.67     0.67
3  Random Forest     macro avg       0.65    0.60      0.60   819.00
8            SVC     macro avg       0.69    0.60      0.58   819.00
4  Random Forest  weighted avg       0.66    0.66      0.63   819.00
9            SVC  weighted avg       0.68    0.67      0.63   819.00